# Python Intro - Final Exercise


In [2]:
import pyodbc
import pandas as pd
import numpy as np
print('ok')

ok


In [3]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [4]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 
courses = pd.read_sql("SELECT * FROM Courses;",conn) 
departments = pd.read_sql("SELECT * FROM Departments;",conn) 
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 



# Questions
 
 
## Q1. Count the number of students on each department

In [5]:
print('\nNumber of students : %d' % students.shape[0])


Number of students : 280


## Q2. How many students have each course of the English department and the total number of students in the department?

In [15]:
courses['DepartmentId'] = courses.DepartmentID

data = pd.merge(pd.merge(pd.merge(pd.merge(classrooms, courses, on='CourseId'), 
                                    departments, on='DepartmentId'), 
                           students, on='StudentId'), 
                  teachers, on='TeacherId')

data = data.rename(columns = {'FirstName_x':'StudentFirstName', 
                         'LastName_x': 'StudentLastName',
                        'Gender_x' : 'StudentGender',
                        'FirstName_y':'TeacherFirstName',
                        'LastName_y':'TeaccherLastName',
                        'Gender_y':'TeacherGender'})

#total student in each department
res = data[['DepartmentName', 'StudentId']].drop_duplicates().groupby('DepartmentName').agg({'StudentId':['count']})

print("Number of students in all departments is\n", res)


#in english department
res1 = data[['DepartmentName', 'StudentId']].drop_duplicates();
print("Number of students in Engllish depsrtment is", res1[res1.DepartmentName == 'English'].shape[0])

data.head()
data[data.DepartmentName == 'English'].groupby(['StudentFirstName','StudentLastName']).agg({'CourseId':['count']})



Number of students in all departments is
                StudentId
                   count
DepartmentName          
Arts                 137
English               84
Science              200
Sport                 81
Number of students in Engllish depsrtment is 84


CourseId
                                    count
StudentFirstName StudentLastName         
 Abbey           Singley                1
 Adrianna        Bean                   1
 Alec            Miller                 1
 Alexandrea      Hannafious             1
 Alexis          Cruse                  1
 Alicia          Schlichting            1
 Alyssa          Fabian                 1
                 Grounds                1
                 Zarazua                1
 Amanda          Thomas                 1
 Andrew          Holiday                1
                 Porter                 1
 Ashleigh        Conder                 1
 Austin          Mottaghi               2
                 Power                  1
 Autumn          Major                  1
 Brighid         Hoenig                 1
 Caitlin         Guthrie                1
 Catherine       Lemme                  1
 Cheyenne        Shawver                1
 Christian       Boucher                1
 Christopher     Patzoldt               1
 Clayton         Rehm                   1
 Cory            Seabolt                1
 Courtney        Shannahan              2
 Cullen          Brewton                1
 Cydney          Petersen               2
 Dakota          Iiams                  1
 Daniel          Thelen-Wade            1
 David           Heffron                1
...                                   ...
 Kira            Dowswell               1
 Kyle            Nibbelink              1
                 Shepler                2
 Laurie          Kelso                  1
 Luke            Taylor                 1
 Malena          Wofford                1
 Maxwell         Kotch                  1
 Melissa         Crespin                2
 Monica          Craig                  2
 Mycah           Bingham                1
 Nicholas        Mckinney               1
 Nichole         Fisher                 1
 Noah            Berzina                1
 Patrick         Hinds                  1
 Rachel          Westcott               1
 Rebecca         Bruch                  1
 Robert          Adams                  1
 Ruben           Mosher                 1
 Sabrina         Mcclure                1
 Samantha        Anderson               1
 Sarah           Bencomo                2
 Sean            Hasenack               1
 Shepherd        Gillham                1
 Sophiya         Aponte                 2
 Tyler           Menard                 2
 Victoria        Kishinevsky            1
 Wyatt           Maus                   1
 Zachary         Haney                  1
                 Storey                 1
                 Yeager                 1

[84 rows x 1 columns]

## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

## Q5. For which courses the percentage of male/female students is over 70%?

## Q6. For each department, how many students passed with a grades over 80?

## Q7. For each department, how many students passed with a grades under 60?

## Q8. Rate the teachers by their average student's grades (in descending order).

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).